In [1]:
import os
import time
import shutil

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

import torch.nn.utils.prune as prune

from models import *
from models.prune_util import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import gc
gc.collect()
torch.cuda.empty_cache()

global best_prec

batch_size = 64
model_name = f"VGG16_os_iter_prune_0.78_q"
fdir = 'result/' + model_name
model = VGG16()
os_prune_vgg16(model, 0.78)
checkpoint = torch.load(f"result/VGG16_os_iter_prune_0.78/model_best.pth.tar")
model.load_state_dict(checkpoint['state_dict'])
model.cuda()

device = torch.device("cuda") 

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))      

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

def train_model(model, fdir, criterion, optimizer, epochs): 
    os.makedirs(fdir, exist_ok=True)

    best_prec = 0

    #model = nn.DataParallel(model).cuda()
    model.cuda()
    criterion = criterion.cuda()
    #cudnn.benchmark = True
            

    for epoch in range(0, epochs):
        adjust_learning_rate(optimizer, epoch)

        train(trainloader, model, criterion, optimizer, epoch)
        
        # evaluate on test set
        print("Validation starts")
        prec = validate(testloader, model, criterion)

        # remember best precision and save checkpoint
        is_best = prec > best_prec
        best_prec = max(prec,best_prec)
        print('best acc: {:1f}'.format(best_prec))
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec': best_prec,
            'optimizer': optimizer.state_dict(),
        }, is_best, fdir)

def val_model(model): 

    model.cuda()
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device) # loading to GPU
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testloader.dataset)

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
            correct, len(testloader.dataset),
            100. * correct / len(testloader.dataset)))

Pruning 50 ic-slices out of 64 ic-slices per output channel (78.1% pruned)
Pruning 50 ic-slices out of 64 ic-slices per output channel (78.1% pruned)
Pruning 100 ic-slices out of 128 ic-slices per output channel (78.1% pruned)
Pruning 100 ic-slices out of 128 ic-slices per output channel (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices per output channel (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices per output channel (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices per output channel (78.1% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)


/tmp/ipykernel_124282/2264729797.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"result/VGG16_os_iter_prune_0.78/model_best.pth.tar")


Files already downloaded and verified
Files already downloaded and verified


In [2]:
val_model(model)


Test set: Accuracy: 9041/10000 (90%)



In [3]:
quantize_pruned(model)

val_model(model)


Test set: Accuracy: 1139/10000 (11%)



In [4]:
lr = 3e-3

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train_model(model, fdir, criterion, optimizer, 100)

Epoch: [0][0/782]	Time 1.061 (1.061)	Data 0.017 (0.017)	Loss 0.0232 (0.0232)	Prec 100.000% (100.000%)
Epoch: [0][100/782]	Time 0.052 (0.078)	Data 0.012 (0.013)	Loss 0.1032 (0.0674)	Prec 93.750% (97.525%)
Epoch: [0][200/782]	Time 0.067 (0.073)	Data 0.012 (0.013)	Loss 0.0320 (0.0678)	Prec 98.438% (97.536%)
Epoch: [0][300/782]	Time 0.066 (0.071)	Data 0.013 (0.013)	Loss 0.2043 (0.0673)	Prec 95.312% (97.612%)
Epoch: [0][400/782]	Time 0.079 (0.070)	Data 0.012 (0.013)	Loss 0.1344 (0.0655)	Prec 95.312% (97.685%)
Epoch: [0][500/782]	Time 0.067 (0.069)	Data 0.012 (0.013)	Loss 0.0308 (0.0641)	Prec 98.438% (97.767%)
Epoch: [0][600/782]	Time 0.067 (0.069)	Data 0.014 (0.013)	Loss 0.0602 (0.0651)	Prec 98.438% (97.775%)
Epoch: [0][700/782]	Time 0.066 (0.069)	Data 0.012 (0.013)	Loss 0.1622 (0.0639)	Prec 93.750% (97.782%)
Validation starts
Test: [0/157]	Time 0.032 (0.032)	Loss 0.4741 (0.4741)	Prec 85.938% (85.938%)
Test: [100/157]	Time 0.031 (0.031)	Loss 0.4337 (0.4838)	Prec 85.938% (89.604%)
 * Prec 90

KeyboardInterrupt: 

In [6]:
model = VGG16()
os_prune_vgg16(model, 0.78)
quantize_pruned(model)

PATH = f"{fdir}/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
model.cuda()

class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []
save_output = SaveOutput()
model.features[40].register_forward_pre_hook(save_output)

val_model(model)

print_sparsity(model)

Pruning 50 ic-slices out of 64 ic-slices per output channel (78.1% pruned)
Pruning 50 ic-slices out of 64 ic-slices per output channel (78.1% pruned)
Pruning 100 ic-slices out of 128 ic-slices per output channel (78.1% pruned)
Pruning 100 ic-slices out of 128 ic-slices per output channel (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices per output channel (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices per output channel (78.1% pruned)
Pruning 200 ic-slices out of 256 ic-slices per output channel (78.1% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)
Pruning 399 ic-slices out of 512 ic-slices per output channel (77.9% pruned)


/tmp/ipykernel_124282/3884249786.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)



Test set: Accuracy: 9034/10000 (90%)

layer 3 sparsity: 0.766
layer 7 sparsity: 0.766
layer 10 sparsity: 0.773
layer 14 sparsity: 0.773
layer 17 sparsity: 0.770
layer 20 sparsity: 0.770
layer 24 sparsity: 0.770
layer 27 sparsity: 0.779
layer 30 sparsity: 0.779
layer 34 sparsity: 0.779
layer 37 sparsity: 0.779
layer 40 sparsity: 0.779


In [14]:
l = model.features[40]
a = [i for i in range(l.weight_mask.size(1)) if l.weight_mask[0,i].sum() > 0]
print(l.weight_q[0,a[:2],:,:])

tensor([[[ 0.0000, -1.4775, -1.4775],
         [ 0.7387,  0.7387,  0.0000],
         [ 0.0000,  0.7387,  0.0000]],

        [[-0.0000,  0.0000,  0.7387],
         [ 0.0000,  0.7387,  1.4775],
         [ 0.7387,  0.7387,  0.7387]]], device='cuda:0',
       grad_fn=<IndexBackward0>)


In [15]:
print(save_output.outputs[0][0][:2,:2])

tensor([[[[0.0000, 0.0807],
          [0.1303, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]]],


        [[[0.5222, 0.3067],
          [0.0775, 0.9398]],

         [[0.0000, 0.0000],
          [0.0000, 0.0000]]]], device='cuda:0')


In [18]:
print(f'Weight int: \n{(model.features[40].weight_q.data / (model.features[40].weight_quant.wgt_alpha.data.item()/(2**(4-1)-1)))[0,a[0]]}')
x = save_output.outputs[0][0]
x_alpha = model.features[40].act_alpha.data.item()
x_delta = x_alpha / (2**(4)-1)
act_q = act_quantization(4)
x_q = act_q(x, x_alpha)
print(f'Act int: \n{(x_q/x_delta)[:2,:2]}')

Weight int: 
tensor([[ 0.0000, -2.0000, -2.0000],
        [ 1.0000,  1.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000]], device='cuda:0')
Act int: 
tensor([[[[0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.]]],


        [[[1., 1.],
          [0., 2.]],

         [[0., 0.],
          [0., 0.]]]], device='cuda:0')
